# Day2 — Relationship Analysis (Scatter + Correlation)

목표:
- 주요 센서간 관계를 산점도로 시각적으로 확인(figures/Day2)
- 상관계수 표 생성(tables/Day2)
- Day3(예측)로 넘어가기 전, 어떤 센서 조합이 의미있는지 1차 판단

## 입력/출력
- 입력(processed): `data/processed/processed_v1_run_abs_features.csv`
- 출력(표): `tables/Day2/`  (corr_all.csv, corr_running.csv)
- 출력(그림): `figures/Day2/` (scatter 4장)

In [ ]:
from pathlib import Path
import pandas as pd

PROCESSED = Path("../../data/processed/processed_v1_run_abs_features.csv")
TABLES_DIR = Path("../../tables/Day2")
FIG_DIR = Path("../../figures/Day2")

PROCESSED.exists(), TABLES_DIR.exists(), FIG_DIR.exists()

## 1) 데이터 로드 & 기본 확인

In [ ]:
df = pd.read_csv(PROCESSED)
df.shape, df.columns[:10]

## 2) 관계 분석 대상 컬럼 확인
- 기본 타겟: FB_Torque
- 비교 대상: FB_Rpm / Vib_rms(대표 진동) / Temp_mean / Press_mean

In [ ]:
cands = ["FB_Torque", "FB_Rpm", "Vib_rms", "Temp_mean", "Press_mean"]
[x for x in cands if x in df.columns]

## 3) Day2 산출물 생성 (스크립트 실행)
- 산점도 4장 저장(figures/Day2)
- 상관표 2개 저장(tables/Day2): 전체 / 운전구간(RPM>0)

노트북에서는 재현성 확인을 위해 스크립트를 호출하고,
생성된 파일이 있는지만 간단히 체크한다.

In [ ]:
import subprocess
import sys

def run(cmd: str):
    print(">", cmd)
    res = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
    return res.returncode

py = sys.executable

# Day2 스크립트 실행(모듈 형태로 호출)
run(f"{py} -m src.day2_relationship")

## 4) 생성 결과 확인
- tables/Day2에 CSV들이 생성되었는지 확인
- figures/Day2에 PNG들이 생성되었는지 확인

In [ ]:
TABLES_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

tables = sorted(TABLES_DIR.glob("*.csv"))
figs = sorted(FIG_DIR.glob("*.png"))

[t.name for t in tables], [f.name for f in figs]

## 5) 상관표 미리보기(요약)
- corr_all: 전체 구간 상관
- corr_running: 운전(RPM>0) 구간 상관

In [ ]:
corr_all = pd.read_csv(TABLES_DIR / "corr_all.csv", index_col=0)
corr_all

In [ ]:
corr_running_path = TABLES_DIR / "corr_running.csv"
if corr_running_path.exists():
    corr_running = pd.read_csv(corr_running_path, index_col=0)
    corr_running
else:
    "corr_running.csv not found"

## 6) Day2 관찰 요약(산점도/상관 기반)
- 토크–RPM 관계가 가장 뚜렷하게 나타남(산점도에서 레짐 분리 + 상관계수도 가장 큼)
- Press_mean / Temp_mean은 값 범위가 좁아 산점도에서 x축이 좁게 보이며, 강한 선형 관계는 약하게 관찰됨
- Vib_rms는 상태(정지/가동) 변화에는 반응하지만, 같은 상태 내에서 토크를 강하게 설명하는 패턴은 제한적

## Day3 계획 (미래 예측)
- 예측 대상 1~2개 선정(FB_Torque 또는 Temp_mean)
- Prophet 기본 모델로 예측 그래프 생성
- 실제와 비교 및 MAE 계산